In [ ]:
from PIL import Image
import numpy as np
from scipy.stats import bernoulli

BLACK = 0
WHITE = 255

def arr_to_bmp(arr, filename = None):
    img = Image.fromarray(arr)
    img = img.resize((10*img.width, 10*img.height), Image.NONE)
    if filename is not None:
        img.save(filename)
    img.show()
    
def results_to_bmp(arr1, arr2, filename = None):
    img1 = Image.fromarray(arr1)
    img2 = Image.fromarray(arr2)
    
    new_width = int((img1.width + img2.width) * 5/4)
    new_height = img1.height
    
    img_res = Image.new("L", (new_width, new_height), 255)
    img_res.paste(img1, (0, 0))
    img_res.paste(img2, (new_width - img2.width, 0))
    img_res = img_res.resize((10*img_res.width, 10*img_res.height), Image.NONE)
    
    if filename is not None:
        img_res.save(filename)
    img_res.show()

def generate_binary_image(size, density = 0.1):
    return bernoulli.rvs(p=1-density, size=(size, size)).astype(np.uint8) * 255

In [ ]:
from itertools import product

def neighs_4(arr_size, x, y):
    return [(x-1, y), (x, y-1), ((x+1) % arr_size, y), (x, (y+1) % arr_size)]

def neighs_8(arr_size, x, y):
    l = [((i + arr_size) % arr_size, (j + arr_size) % arr_size) for (i,j) in product(range(x-1,x+2), range(y-1, y+2))]
    l.remove((x, y))
    return l
    
def neighs_8_16(arr_size, x, y):
    l = [((i + arr_size) % arr_size, (j + arr_size) % arr_size) for (i,j) in product(range(x-2,x+3), range(y-2, y+3))]
    l.remove((x, y))
    return l

def to_recalculate(arr_size, x, y, neigh_func):
    s = set()
    neighs = neigh_func(arr_size, x, y)
    for (n_x, n_y) in neighs:
        for p in neigh_func(arr_size, n_x, n_y):
            s.add(p)
    s.add((x, y))
    return s

In [ ]:
def magnet_energy(arr, arr_size, i, j, neighs_func):
    E = 0
    i = i % arr_size
    j = j % arr_size
    neighs = neighs_func(arr_size, i, j)
    for x, y in neighs:
        if arr[x, y] == arr[i, j]:
            E += 1
        else:
            E -= 1
    return E

def magnet_energy_inv(arr, arr_size, i, j, neighs_func):
    E = 0
    i = i % arr_size
    j = j % arr_size
    neighs = neighs_func(arr_size, i, j)
    for x, y in neighs:
        if arr[x, y] == arr[i, j]:
            E -= 1
        else:
            E += 1
    return E
    
def arr_energy(arr, arr_size, point_energy, neighs_func):
    en_arr = np.zeros((arr_size, arr_size))
    E = 0
    for i in range(arr_size):
        for j in range(arr_size):
            en_arr[i, j] = point_energy(arr, arr_size, i, j, neighs_func)
            E += en_arr[i,j]
    return en_arr, E

In [ ]:
NEIGH_MODE = False
RAND_MODE = True

def neigh_rand(arr, arr_size):
    while True:
        row = np.random.randint(0, arr_size)
        col = np.random.randint(0, arr_size)

        off = np.random.randint(0,2)
        row2 = (row + off) % arr_size
        col2 = (col + 1 - off) % arr_size
        if arr[row, col] != arr[row2, col2]:
            break
    return ((row, col), (row2, col2))

def rand_rand(arr, arr_size):
    while True:
        row = np.random.randint(0, arr_size)
        col = np.random.randint(0, arr_size)

        row2 = np.random.randint(0, arr_size)
        col2 = np.random.randint(0, arr_size)
        if arr[row, col] != arr[row2, col2]:
            break
    return ((row, col), (row2, col2))

def sa_bit_image(in_arr, 
                 Tmax = 10, 
                 Tmin = 0.01, 
                 Tchange=lambda T: 0.999*T, 
                 point_energy = magnet_energy, 
                 neighs_func = neighs_4, 
                 neigh_gen = neigh_rand,
                 iters = None
                ):
    T = Tmax
    arr = in_arr.copy()
    arr_size = len(arr)
    (en_arr, whole_energy) = arr_energy(arr, arr_size, point_energy, neighs_func)
    iteration = 0
    stop_cond = False
    while not stop_cond:
        new_energy = whole_energy
        ((row1, col1), (row2, col2)) = neigh_gen(arr, arr_size)

        arr[row1, col1], arr[row2, col2] = arr[row2, col2], arr[row1, col1]

        recalc_set = to_recalculate(arr_size, row1, col1, neighs_func) | to_recalculate(arr_size, row2, col2, neighs_func)

        for x, y in recalc_set:
            new_energy -= en_arr[x, y]
            new_energy += point_energy(arr, arr_size, x, y, neighs_func)

        delta_energy = new_energy - whole_energy
        if delta_energy <= 0 or bernoulli.rvs(p=np.exp(-delta_energy/T)) == 1:
            whole_energy = new_energy
            for x, y in recalc_set:
                en_arr[x, y] = point_energy(arr, arr_size, x, y, neighs_func)
        else:
            arr[row1, col1], arr[row2, col2] = arr[row2, col2], arr[row1, col1] 
            
        if iters is not None:
            iteration += 1
            stop_cond = (iteration >= iters)
        else:
            stop_cond = (T < Tmin)
            
        T = Tchange(T)
    return arr

# Wyniki działania dla różnych gęstości

In [ ]:
dens_arr01 = generate_binary_image(40, 0.1)
dens_arr03 = generate_binary_image(40, 0.3)
dens_arr04 = generate_binary_image(40, 0.4)

dens_arr01_res = sa_bit_image(dens_arr01)
dens_arr03_res = sa_bit_image(dens_arr03)
dens_arr04_res = sa_bit_image(dens_arr04)

results_to_bmp(dens_arr01, dens_arr01_res, "dens01.png")
results_to_bmp(dens_arr03, dens_arr03_res, "dens03.png")
results_to_bmp(dens_arr04, dens_arr04_res, "dens04.png")

# Wyniki dla różnych rodzajów sąsiedztwa i funkcji energii

In [ ]:
neigh_arr = generate_binary_image(40, 0.4)

neigh4_res_mag = sa_bit_image(neigh_arr, neighs_func=neighs_4, point_energy=magnet_energy)
neigh8_res_mag = sa_bit_image(neigh_arr, neighs_func=neighs_8, point_energy=magnet_energy)
neigh8_16_res_mag = sa_bit_image(neigh_arr, neighs_func=neighs_8_16, point_energy=magnet_energy)

neigh4_res_inv = sa_bit_image(neigh_arr, neighs_func=neighs_4, point_energy=magnet_energy_inv)
neigh8_res_inv = sa_bit_image(neigh_arr, neighs_func=neighs_8, point_energy=magnet_energy_inv)
neigh8_16_res_inv = sa_bit_image(neigh_arr, neighs_func=neighs_8_16, point_energy=magnet_energy_inv)

arr_to_bmp(neigh_arr, "neigh.png")

results_to_bmp(neigh4_res_mag, neigh4_res_inv, "neigh4.png")
results_to_bmp(neigh8_res_mag, neigh8_res_inv, "neigh8.png")
results_to_bmp(neigh8_16_res_mag, neigh8_16_res_inv, "neigh8_16.png")

# Porównanie dla różnych szybkości spadku energii

In [ ]:
ener_arr = generate_binary_image(40, 0.4)

ener09_res = sa_bit_image(ener_arr, Tchange=lambda T: 0.9*T)
ener099_res = sa_bit_image(ener_arr, Tchange=lambda T: 0.99*T)
ener0999_res = sa_bit_image(ener_arr, Tchange=lambda T: 0.999*T)

results_to_bmp(ener_arr, ener09_res, "ener09.png")
results_to_bmp(ener_arr, ener099_res, "ener099.png")
results_to_bmp(ener_arr, ener0999_res, "ener0999.png")

# Porównanie dla różnych sposobów generacji stanów sąsiednich

In [ ]:
next_arr = generate_binary_image(40, 0.4)

next4_res_n = sa_bit_image(next_arr, neighs_func=neighs_4, neigh_gen=neigh_rand)
next8_res_n = sa_bit_image(next_arr, neighs_func=neighs_8, neigh_gen=neigh_rand)
next8_16_res_n = sa_bit_image(next_arr, neighs_func=neighs_8_16, neigh_gen=neigh_rand)

next4_res_r = sa_bit_image(next_arr, neighs_func=neighs_4, neigh_gen=rand_rand)
next8_res_r = sa_bit_image(next_arr, neighs_func=neighs_8, neigh_gen=rand_rand)
next8_16_res_r = sa_bit_image(next_arr, neighs_func=neighs_8_16, neigh_gen=rand_rand)

arr_to_bmp(next_arr, "next.png")

results_to_bmp(next4_res_n, next4_res_r, "next4.png")
results_to_bmp(next8_res_n, next8_res_r, "next8.png")
results_to_bmp(next8_16_res_n, next8_16_res_r, "next8_16.png")